In [ ]:
from requests import get
from bs4 import BeautifulSoup
from IPython.core.display import clear_output
from time import sleep, time
from random import randint
import warnings
    

In [121]:
average_ratings = {}
number_of_reviews = {}
price_per_person = {}

veg_restaurants_links = {
    "La Taqueria": "https://www.yelp.com/biz/la-taqueria-san-francisco-2?osq=restaurants",
    "Curry Up Now": "https://www.yelp.com/biz/curry-up-now-san-francisco-5?osq=restaurants",
    "Lucky Creation": "https://www.yelp.com/biz/lucky-creation-vegetarian-restaurant-san-francisco?osq=vegetarian",
    "Shizen Vegan Sushi Bar & Izakaya": "https://www.yelp.com/biz/shizen-vegan-sushi-bar-and-izakaya-san-francisco?osq=restaurants",
    "Greens Restaurant": "https://www.yelp.com/biz/greens-restaurant-san-francisco-5?osq=restaurants",
    "Nightbird": "https://www.yelp.com/biz/nightbird-san-francisco?osq=Restaurants",
    "Gracias Madre": "https://www.yelp.com/biz/gracias-madre-san-francisco?osq=restaurants",
    "Maruya": "https://www.yelp.com/biz/maruya-san-francisco?osq=Restaurants",
    "The Dosa Brothers": "https://www.yelp.com/biz/the-dosa-brothers-san-francisco-10?osq=Restaurants",
    "Cha-Ya": "https://www.yelp.com/biz/cha-ya-san-francisco-5?osq=restaurants",
    "Udupi Palace": "https://www.yelp.com/biz/udupi-palace-san-francisco?osq=restaurants",
    "Peasant Pies Noe Valley": "https://www.yelp.com/biz/peasant-pies-noe-valley-san-francisco?osq=Restaurants",
    "Beharam": "https://www.yelp.com/biz/besharam-san-francisco?osq=restaurants",
    "Lucky Creation Vegeterian Restaurant": "https://www.yelp.com/biz/lucky-creation-vegetarian-restaurant-san-francisco?osq=Restaurants",
    "Beloved Cafe": "https://www.yelp.com/biz/beloved-cafe-san-francisco?osq=restaurants",
    "Morty's Delicatessen": "https://www.yelp.com/biz/mortys-delicatessen-san-francisco?osq=Restaurants",
    "Donut Farm": "https://www.yelp.com/biz/donut-farm-san-francisco?osq=Restaurants",
    "MIXT": "https://www.yelp.com/biz/mixt-san-francisco-13?osq=restaurants",
    "Something Simple for Lunch": "https://www.yelp.com/biz/something-simple-for-lunch-san-francisco?osq=Restaurants",
    "Indochine Vegan": "https://www.yelp.com/biz/indochine-vegan-san-francisco-2?osq=restaurants",
    
}


In [122]:
## find average rating of each restaurant
def scrape_average_ratings(rating_container):
    ## some links have different span className
    if rating_container is None:
        rating_container = html_soup.find('div', class_ = "lemon--div__373c0__1mboc i-stars__373c0__Y2F3O i-stars--large-4-half__373c0__2rd5j border-color--default__373c0__2oFDT overflow--hidden__373c0__8Jq2I")
        if rating_container is None:
            rating_container = html_soup.find('div', class_ = "lemon--div__373c0__1mboc i-stars__373c0__Y2F3O i-stars--large-3-half__373c0__3i1MU border-color--default__373c0__2oFDT overflow--hidden__373c0__8Jq2I")
            return rating_container['aria-label']
        else:
            return rating_container['aria-label']
    else:
        return rating_container['aria-label']

        
## find number of ratings of each restaurant
def scrape_number_of_reviews(review_container):
    return review_container.p.text

## find price per person of each restaurant
def scrape_price_per_person(html_soup):
    price_container = html_soup.find('span', class_ = "lemon--span__373c0__3997G text__373c0__2pB8f text-color--normal__373c0__K_MKN text-align--left__373c0__2pnx_ text-bullet--after__373c0__1ZHaA text-size--large__373c0__1568g")
    return price_container.text

def scrape_reviews(html_soup):
    review_container = html_soup.find_all('li', class_ = "lemon--li__373c0__1r9wz u-space-b3 u-padding-b3 border--bottom__373c0__uPbXS border-color--default__373c0__2oFDT")

In [124]:
start_time = time()
requests = 0

for i in veg_restaurants_links:
    response = get(veg_restaurants_links[i])
    sleep(randint(8,15))
    
    requests += 1
    elapsed_time = time() - start_time
    print("Request {}, Frequency {} requests/s".format(requests, requests/elapsed_time))
    clear_output(wait=True)
    
    html_soup = BeautifulSoup(response.text, 'html.parser')
    ## this container contains both review and rating containers
    review_and_rating_container = html_soup.find('div', class_ = "lemon--div__373c0__1mboc arrange__373c0__UHqhV gutter-6__373c0__zqA5A vertical-align-middle__373c0__2TQsQ u-space-b1 border-color--default__373c0__2oFDT")
    
    ## find asverage rating and numbre of reviews of each restaurant
    rating_container = review_and_rating_container.find('div', class_ = "lemon--div__373c0__1mboc i-stars__373c0__Y2F3O i-stars--large-4__373c0__3fk-p border-color--default__373c0__2oFDT overflow--hidden__373c0__8Jq2I")
    review_container = review_and_rating_container.find('div', class_ = "lemon--div__373c0__1mboc arrange-unit__373c0__1piwO border-color--default__373c0__2oFDT nowrap__373c0__1_N1j")
    average_ratings[i] = scrape_average_ratings(rating_container)
    number_of_reviews[i] = scrape_number_of_reviews(review_container)
    
    price_per_person[i] = scrape_price_per_person(html_soup)



Request 1, Frequency 0.06307639605550555 requests/s


In [125]:
print(number_of_reviews)

{'La Taqueria': '3987 reviews'}


In [108]:
def clean_data():
    for rating in average_ratings:
        average_ratings[rating] = float(average_ratings[rating].replace(" star rating", ""))
    
    for i in number_of_reviews:
        number_of_reviews[i] = int(number_of_reviews[i].replace(" reviews", ""))
    
    for i in price_per_person:
        price_per_person[i] = price_per_person[i].replace(" ", "")

In [120]:
average_ratings

{'Curry Up Now': 3.5,
 'Lucky Creation': 4.0,
 'Shizen Vegan Sushi Bar & Izakaya': 4.5,
 'Greens Restaurant': 4.0,
 'Nightbird': 4.0,
 'Gracias Madre': 4.0,
 'Maruya': 4.0,
 'The Dosa Brothers': 4.5,
 'Cha-Ya': 4.0,
 'Udupi Palace': 4.0,
 'Peasant Pies Noe Valley': 4.0,
 'Beharam': 4.0,
 'Lucky Creation Vegeterian Restaurant': 4.0,
 'Beloved Cafe': 4.5,
 "Morty's Delicatessen": 4.0,
 'Donut Farm': 4.0,
 'MIXT': 4.0}